In [1]:
from lg_capstone_2024.describe_module import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [2]:
# 범주형 변수를 인코딩하는 함수
def encode_categorical_features(data):
    label_encoders = {}
    for column in data.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column].astype(str))  # 문자열로 변환 후 인코딩
        label_encoders[column] = le
    return data, label_encoders


# 정규화 함수
def normalize_data(data):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data.select_dtypes(include=[np.number]))  # 수치형 데이터만 정규화
    scaled_df = pd.DataFrame(scaled_data, columns=data.select_dtypes(include=[np.number]).columns, index=data.index)
    return scaled_df

In [3]:
all_data = pd.read_csv("data/all_data.csv")

C:\Users\user\AppData\Local\Temp\ipykernel_20980\978275038.py:1: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv("data/all_data.csv")


In [4]:
# target 값이 비어있지 않은 데이터(Train data)
train_data = all_data[all_data['target'].notnull()]
train_data = train_data.drop(['Set ID'], axis=1)    
# target 값이 비어있는 데이터(Test data)
test_data = all_data[all_data['target'].isnull()]
test_data = test_data.drop(['Set ID'], axis=1)    

In [5]:
# True -> Train data, False -> Test data
train_or_test = False

In [10]:
# 결과 확인
print(f"Train Data Shape: {train_data.shape}")
print(f"Test Data Shape: {test_data.shape}")

Train Data Shape: (40506, 60)
Test Data Shape: (17361, 60)


In [12]:
# 결측값이 있는 행 삭제
#test는 결측값이 있는 행이 없음
if(train_or_test):
    train_data_cleaned = train_data.dropna()


# target 값 추출
if(train_or_test):
    y = train_data_cleaned['target']
    train_data_cleaned = train_data_cleaned.drop(['target'], axis=1)
else:
    test_data_cleaned = test_data.drop(['target'], axis=1)


(17361, 59)


In [13]:
### 1단계: 범주형 데이터 인코딩
if(train_or_test):
    train_data_encoded, label_encoders = encode_categorical_features(train_data_cleaned) #이후 Inversion Transform 메서드를 사용하기 위해 label_encoders를 저장
else:
    test_data_encoded, label_encoders = encode_categorical_features(test_data_cleaned) #이후 Inversion Transform 메서드를 사용하기 위해 label_encoders를 저장
    print(test_data_encoded.shape)

# 정규화 진행
if(train_or_test):
    train_data_scaled = normalize_data(train_data_encoded)
else:
    test_data_scaled = normalize_data(test_data_encoded)

# 원본 데이터와 결합 (인코딩된 범주형 데이터는 함께 포함)
if(train_or_test):
    train_data_final = pd.concat([train_data_scaled, train_data_encoded.select_dtypes(include=['object'])], axis=1)
else:
    test_data_final = pd.concat([test_data_scaled, test_data_encoded.select_dtypes(include=['object'])], axis=1)

(17361, 59)


In [14]:
# 데이터 저장하기
if(train_or_test):
    #train_data_final csv 파일로 저장
    train_data_final.to_csv("data/train_data_final.csv", index=False)
    y.to_csv("data/train_y.csv", index=False)
else:
    #test_data_final csv 파일로 저장
    test_data_final.to_csv("data/test_data_final.csv", index=False)